In [ ]:
pip install tensorflow numpy scikit-learn

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

# Select only 20 classes
selected_classes = list(range(20))  # you can customize this list
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train = x_train[train_mask]
y_train = y_train[train_mask]
x_test = x_test[test_mask]
y_test = y_test[test_mask]

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Re-map class labels to range 0-19
class_mapping = {original: new for new, original in enumerate(selected_classes)}
y_train = np.vectorize(class_mapping.get)(y_train)
y_test = np.vectorize(class_mapping.get)(y_test)

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Resize images to match VGG16 input size
x_train = tf.image.resize(x_train, (224, 224))
x_test = tf.image.resize(x_test, (224, 224))

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=20)
y_test = to_categorical(y_test, num_classes=20)

In [ ]:
# Load VGG16 model without top
base_model = Xception(include_top=False,
                      weights="imagenet",
                      input_tensor=None,
                      input_shape=None,
                      pooling=None,
                      classes=1000,
                      classifier_activation="softmax",
                      name="xception")

# Freeze base model layers
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(20, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 61s 174ms/step - accuracy: 0.4749 - loss: 2.0589 - val_accuracy: 0.7635 - val_loss: 0.8276
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 75s 158ms/step - accuracy: 0.7953 - loss: 0.7162 - val_accuracy: 0.7935 - val_loss: 0.6801
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 79s 150ms/step - accuracy: 0.8245 - loss: 0.5831 - val_accuracy: 0.8095 - val_loss: 0.6201
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 158ms/step - accuracy: 0.8368 - loss: 0.5198 - val_accuracy: 0.8180 - val_loss: 0.5907
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 79s 150ms/step - accuracy: 0.8608 - loss: 0.4533 - val_accuracy: 0.8220 - val_loss: 0.5760
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 150ms/step - accuracy: 0.8739 - loss: 0.4139 - val_accuracy: 0.8235 - val_loss: 0.5709
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 150ms/step - accuracy: 0.8890 - loss: 0.3808 - val_accuracy: 0.8190 - val_loss: 0.5634
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 155ms/step - accuracy: 0.8946 - loss: 0